In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [6]:
# 2. Create a temporary view of the DataFrame.
# Load the CSV file into a DataFrame
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the DataFrame to verify it's loaded correctly
df.show()

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# Perform the query with the correct column name
query = """
SELECT
    YEAR(date) AS year_sold,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    year_sold
ORDER BY
    year_sold
"""

# Run the query and show the results
avg_price_df = spark.sql(query)
avg_price_df.show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# SQL query to find the average price of homes with 3 bedrooms and 3 bathrooms for each year built
query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    year_built
ORDER BY
    year_built
"""

# Run the query and show the results
avg_price_df = spark.sql(query)
avg_price_df.show()




+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [13]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# SQL query to find the average price of homes with specific criteria for each year built
query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    year_built
ORDER BY
    year_built
"""

# Run the query and show the results
avg_price_df = spark.sql(query)
avg_price_df.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# Start timing
start_time = time.time()

# SQL query to find the view rating for the average price of homes priced at $350,000 or more
query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000
GROUP BY
    view
ORDER BY
    view
"""

# Run the query
view_avg_price_df = spark.sql(query)
view_avg_price_df.show()

# End timing and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 0.8458645343780518 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.
# Cache the temporary table
spark.catalog.cacheTable("home_sales")

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# Start timing for cached query
start_time_cached = time.time()

# SQL query to find the view rating for the average price of homes priced at $350,000 or more
query_cached = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000
GROUP BY
    view
ORDER BY
    view
"""

# Run the query on cached data
view_avg_price_df_cached = spark.sql(query_cached)
view_avg_price_df_cached.show()

# End timing for cached query and print the runtime
print("--- Cached Query Runtime: %s seconds ---" % (time.time() - start_time_cached))



+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- Cached Query Runtime: 1.957737922668457 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Write the DataFrame to Parquet format, partitioned by 'date_built'
df.write.partitionBy("date_built").parquet("/content/home_sales_parquet")

In [20]:
# 11. Read the parquet formatted data.
# Read the Parquet formatted data
parquet_df = spark.read.parquet("/content/home_sales_parquet")

# Show the DataFrame to verify it's loaded correctly
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [21]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [22]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
# Start timing for the query on Parquet data
start_time_parquet = time.time()

# SQL query to find the view rating for the average price of homes priced at $350,000 or more
query_parquet = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales_parquet
WHERE
    price >= 350000
GROUP BY
    view
ORDER BY
    view
"""

# Run the query on the Parquet data
view_avg_price_df_parquet = spark.sql(query_parquet)
view_avg_price_df_parquet.show()

# End timing for the query on Parquet data and print the runtime
print("--- Parquet Query Runtime: %s seconds ---" % (time.time() - start_time_parquet))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- Parquet Query Runtime: 2.097557306289673 seconds ---


In [25]:
# 14. Uncache the home_sales temporary table.
# Uncache the home_sales temporary table
spark.catalog.uncacheTable("home_sales")

False


In [26]:
# 15. Check if the home_sales is no longer cached
print(spark.catalog.isCached("home_sales"))  # This should return False

False
